# Scenario 1: Calibrating models and estimating causal effects

In this scenario, we are interested in determining the effects of masking and social distancing on Covid-19 infections using simulated data for four different countries: Afghanistan, Colombia, France, and the United Kingdom. The simulations use contact matrices and populations subdivided into age groups for each of the countries. In these simulations, each country implements interventions at the same time, and we are interested in understanding how the effects of the interventions differ between countries. The data are generated from an SEIRD model. 

In these questions, we provide the contact matrices and population data for each country, as well as the outputs of the simulated SEIRD model. We ask you to calibrate a model to estimate R0, and to estimate the causal effects of interventions.

In the data provided, ‘UK_population.csv’ and ‘UK_contact_matrix.csv’ contain data on the population counts and the contact matrix for each age group in the UK. ‘UK_compartments.csv’ has compartments S, E, ICase, IMild, R, and D. All patients in IMild transition to R, whereas all patients in ICase transition to the hospital (IHospital, not currently included in this output). From there, some patients recover and others transition to D. ‘UK_infections.csv’ has total infections for each day. There are analogous files for the other countries.

In each country:
1. From t = 0 to t = 40 days, there are no interventions in place, and Covid spreads unabated.
2. At t = 40 days, people begin wearing masks, decreasing the spread of covid.
3. At t = 100 days, mask fatigue sets in. People start to mask less, but more than they did from t = 0 to t = 40. Cases increase again.

During this same time frame, each country begins implementing a social distancing policy at t = 60 days. Due to the policy, contact decreases to 30% of its original levels for the interactions of people over the age of 65 with others over the age of 65 (i.e., the 3x3 matrix on the bottom right of the contact matrix) and it decreases to 60% of its original levels otherwise.

Because this model is stochastic, (in particular, the number of people transitioning between compartments is randomly drawn from a binomial distribution), we run each simulation ten times for each country. The dt parameter is set to 0.2 so that there are 5 time steps per day and 1000 time steps in each 200-day simulation.

Using the data provided, which includes contact matrices and population counts for all four countries, along with the output from running 10 simulations for each country:

### (1) Calibrate models to estimate R0 for each country in each of the four time intervals: [0,40), [40,60), [60,100), [100, 200].

### (2) Estimate the causal effects of masking alone, the combined effect of masking and social distancing, and the effect of social distancing alone on infections. Include uncertainty in the estimated effects.

### (3) (Optional) In each country, what is the maximum value R0  can be over the last time interval ([100, 200]), to ensure that there are no more than 50 infections at t = 200 days with 90% confidence? Are these different across countries? 

### (4) (Optional) Can R0 be changed in one of the three preceding intervals ([0,40), [40,60), [60,100)), without changing it in the fourth, to ensure that infections at t = 200 days stay below ¾ of their simulated value at t = 200 with 95% confidence?"

### Load dependencies

In [1]:
import os
import pyciemss
import torch
import pandas as pd
import numpy as np
import requests
from io import StringIO
from typing import Dict, List

import pyciemss.visuals.plots as plots
import pyciemss.visuals.vega as vega
import pyciemss.visuals.trajectories as trajectories

# Country 1: Let's start with France

### Collect data

In [2]:
DATA_PATH = "https://raw.githubusercontent.com/ciemss/program-milestones/epi-scenario-1/18-month-milestone/hackathon/epi/Scenario%201%20Supplemental/"

COUNTRY = "France" # "UK", "Afghanistan", "Colombia"
infection_data = os.path.join(DATA_PATH, COUNTRY + "_infections.csv")
death_data = os.path.join(DATA_PATH, COUNTRY + "_compartments.csv")
population_data = os.path.join(DATA_PATH, COUNTRY + "_population.csv")
contact_matrix = os.path.join(DATA_PATH, COUNTRY + "_contact_matrix.csv")

### Define functions to read and manipulate data

In [3]:
def read_data(data_location: str):
    
    # Fetch the contents of the CSV file from the URL
    response = requests.get(data_location)
    csv_content = StringIO(response.text)
    df = pd.read_csv(csv_content)
    # print(df.head())
    
    return df

def add_over_age_groups(df: pd.DataFrame, input_comp: str, output_comp: str):

    # Aggregate data by age
    df = df.drop(columns=['Unnamed: 0'])
    replicate_1_df = df[df['replicate'] == 1] # only using first run
    compartment_df = replicate_1_df[replicate_1_df["compartment"] == input_comp] # only use chosen compartment
    result = compartment_df.groupby('t')['y'].sum().reset_index()
    result.rename(columns={'t': 'Timestamp', 'y': output_comp}, inplace=True)
    result.set_index('Timestamp', inplace=True)
    result.index.name = 'Timestamp'
    # print(result.head())

    return result

### Produce dataset for calibration, calculate total population

In [4]:
# Aggregate case data by age
case_by_age_df = read_data(infection_data)
data_df = add_over_age_groups(case_by_age_df, "infections", "cases")

# # Aggregate death data by age
# death_by_age_df = read_data(death_data)
# death_df = add_over_age_groups(death_by_age_df, "D", "deaths")

# # Merge data into one DataFrame
# death_column = death_df["deaths"]
# data_df = pd.concat([data_df, death_column], axis=1)

# data_df.to_csv(COUNTRY + "_case_death_data.csv", index=True)
data_df.to_csv(COUNTRY + "_case_data.csv", index=True)

# Calculate the total population
pop_data_by_age = read_data(population_data)
total_population = pop_data_by_age["n"].sum()
print(COUNTRY + " total population:", total_population)

France total population: 65273512


### Create base model
This code is adapted from https://github.com/gyorilab/mira/blob/main/notebooks/hackathon_2024.02/scenario1/epi_scenario1.ipynb

### Load mira dependencies

In [5]:
# General imports
import sympy
import json
from mira.metamodel import *
from mira.examples.concepts import susceptible, exposed, infected, recovered, dead
from mira.modeling import Model
from mira.modeling.amr.petrinet import AMRPetriNetModel, template_model_to_petrinet_json
from mira.metamodel.io import model_to_json_file, model_from_json_file
from pathlib import Path

In [21]:
# Build base model
person_units = lambda: Unit(expression=sympy.Symbol('person'))
day_units = lambda: Unit(expression=sympy.Symbol('day'))
per_day_units = lambda: Unit(expression=1/sympy.Symbol('day'))
dimensionless_units = lambda: Unit(expression=sympy.Integer('1'))
per_day_per_person_units = lambda: Unit(expression=1/(sympy.Symbol('day')*sympy.Symbol('person')))

_infected = Concept(name='I', units=person_units(), identifiers={'ido': '0000511'})

c = {
    'S': Concept(name='S', units=person_units(), identifiers={'ido': '0000514'}),
    'E': Concept(name='E', units=person_units(), identifiers={'apollosv': '00000154'}),
    'I_mild': _infected.with_context(status="mild"),
    'I_case': _infected.with_context(status="case"),
    'R': Concept(name='R', units=person_units(), identifiers={'ido': '0000592'}),
    'I_hosp': _infected.with_context(status="hosp"),
    'D': Concept(name='D', units=person_units(), identifiers={'ncit': 'C28554'}),
}
c['I_case'].name = 'I_case'
c['I_mild'].name = 'I_mild'
c['I_hosp'].name = 'I_hosp'


parameters = {
    'beta_mild': Parameter(name='beta_mild', units=per_day_units()),  # Infection rate from I_mild
    'beta_case': Parameter(name='beta_case', units=per_day_units()),  # Infection rate from I_case
    'alpha_mild': Parameter(name='alpha_mild', units=per_day_units()),  # Rate of progressing E -> I_mild
    'alpha_case': Parameter(name='alpha_case', units=per_day_units()),  # Rate of progressing E -> I_case
    'gamma_mildR': Parameter(name='gamma_mildR', units=per_day_units()),  # Rate of progressing I_mild -> R
    'gamma_caseH': Parameter(name='gamma_caseH', units=per_day_units()),  # Rate of progressing I_case -> I_hosp
    'delta_caseR': Parameter(name='delta_caseR', units=per_day_units()),  # Rate of progressing I_hosp -> R
    'mu_caseD': Parameter(name='mu_caseD', units=per_day_units()),  # Rate of progressing I_hosp -> D
    'N': Parameter(name='N', value=total_population, units=person_units()), 
}

S, E, I_mild, I_case, I_hosp, R, D, N, t_0, t, beta_mild, beta_case, alpha_mild, alpha_case, gamma_mildR, gamma_caseH, delta_caseR, mu_caseD = \
    sympy.symbols(
        'S E I_mild I_case I_hosp R D N t_0 t beta_mild beta_case alpha_mild alpha_case gamma_mildR gamma_caseH delta_caseR mu_caseD'
    )


# S -> E by I_mild
exposure_mild = ControlledConversion(
    subject=c['S'],
    outcome=c['E'],
    controller=c['I_mild'],
    rate_law=S*I_mild*beta_mild / N
)
# S -> E by I_case
exposure_case = ControlledConversion(
    subject=c['S'],
    outcome=c['E'],
    controller=c['I_case'],
    rate_law=S*I_case*beta_case / N
)

# E -> I_mild
infection_mild = NaturalConversion(
    subject=c['E'],
    outcome=c['I_mild'],
    rate_law=alpha_mild*E
)
# E -> I_case
infection_case = NaturalConversion(
    subject=c['E'],
    outcome=c['I_case'],
    rate_law=alpha_case*E
)

# I_mild -> R
recovery_mild = NaturalConversion(
    subject=c['I_mild'],
    outcome=c['R'],
    rate_law=gamma_mildR*I_mild
)
# I_case -> I_hosp
hospitalized_case = NaturalConversion(
    subject=c['I_case'],
    outcome=c['I_hosp'],
    rate_law=gamma_caseH*I_case
)
# I_hosp -> R
recovery_case = NaturalConversion(
    subject=c['I_hosp'],
    outcome=c['R'],
    rate_law=delta_caseR*I_hosp
)
# I_hosp -> D
death_case = NaturalConversion(
    subject=c['I_hosp'],
    outcome=c['D'],
    rate_law=mu_caseD*I_hosp
)

seiirhd_model = TemplateModel(
    templates=[
        exposure_mild,
        exposure_case,
        infection_mild,
        infection_case,
        recovery_mild,
        hospitalized_case,
        recovery_case,
        death_case,
    ],
    parameters=parameters,
    time=Time(name='t', units=day_units()),
    annotations=Annotations(name='EPI Scenario 1 France')
)

# Save as JSON
model_to_json_file(seiirhd_model, 'scenario1_base.json')

with open('SEIIRHD_model_France.json', 'w') as fh:
    json.dump(template_model_to_petrinet_json(seiirhd_model), fh, indent=1)

In [ ]:
### Set model path

In [6]:
# MODEL_PATH = "/18-month-milestone/hackathon/epi/scenario1/ciemss/"

# model1 = os.path.join(MODEL_PATH, "SEIIRHD_model_" + COUNTRY + ".json")

model1 = "https://raw.githubusercontent.com/ciemss/program-milestones/epi-scenario-1/18-month-milestone/hackathon/epi/scenario1/ciemss/SEIIRHD_model_France.json"

## Simulate model

In [7]:
start_time = 0.0
end_time = 100.0
logging_step_size = 10.0
num_samples = 10

In [8]:
result1 = pyciemss.sample(model1, end_time, logging_step_size, num_samples, start_time=start_time)
display(result1['data'].head())

# Plot results for all states
schema = plots.trajectories(result1["data"], keep=".*_state")
plots.save_schema(schema, "_schema.json")
plots.ipy_display(schema, dpi=150)

ERROR:root:
                ###############################

                There was an exception in pyciemss

                Error occured in function: sample

                Function docs : 
    Load a model from a file, compile it into a probabilistic program, and sample from it.

    Args:
        model_path_or_json: Union[str, Dict]
            - A path to a AMR model file or JSON containing a model in AMR form.
        end_time: float
            - The end time of the sampled simulation.
        logging_step_size: float
            - The step size to use for logging the trajectory.
        num_samples: int
            - The number of samples to draw from the model.
        solver_method: str
            - The method to use for solving the ODE. See torchdiffeq's `odeint` method for more details.
            - If performance is incredibly slow, we suggest using `euler` to debug.
              If using `euler` results in faster simulation, the issue is likely that the model is s

JSONDecodeError: Expecting value: line 454 column 4 (char 9096)